<a href="https://colab.research.google.com/github/Ming3993/IBD-Homework/blob/main/prepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Java and Spark on Hadoop

In [4]:
# install java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# install spark (change the version number if needed)
!wget -q https://downloads.apache.org/spark/spark-3.5.6/spark-3.5.6-bin-hadoop3.tgz
# unzip the spark file to the current folder
!tar xf spark-3.5.6-bin-hadoop3.tgz

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [5]:
# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.6-bin-hadoop3"

# Create a SparkSession in Python

In [6]:
# start pyspark
!pip install findspark
import findspark
findspark.init()

In [7]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Spark + MongoDB + GraphFrames") \
    .master("local[*]") \
    .config("spark.mongodb.input.uri", mongo_uri) \
    .config("spark.mongodb.output.uri", mongo_uri) \
    .config(
        "spark.jars.packages",
        "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1,"
        "graphframes:graphframes:0.8.4-spark3.5-s_2.12"
    ) \
    .getOrCreate()

NameError: name 'mongo_uri' is not defined

# Example 1: WordCount with Spark DataFrames and Spark RDDs




In [ ]:
# Load the data
!git clone https://github.com/nnthaofit/CSC14118.git

### Spark DataFrame-based WordCount

In [ ]:
linesDF = spark.read.text("CSC14118/ppap.txt")
linesDF.show(linesDF.count(),truncate = False)

from pyspark.sql import functions as f
wordsDF = linesDF.withColumn("word", f.explode(f.split(f.col("value"), " ")))\
    .groupBy("word")\
    .count()\
    .sort("count", ascending = False)
wordsDF.show()

###RDD-based WordCount

In [ ]:
linesRdd = spark.sparkContext.textFile("CSC14118/ppap.txt")
wordsRdd = linesRdd.flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b)\
    .sortBy(lambda pair:-1*pair[1])
wordsRdd.collect()

# Exercise 1: Data query with Spark DataFrame

In [ ]:
# clone the example data files from GitHub to Drive
!git clone https://github.com/nnthaofit/CSC14118.git

###0. Load the data file: movies.json

### 1a. Show the schema of DataFrame that stores the movies dataset.

### 1b. Show the number of distinct movies in the dataset

### 2. Count the number of movies released during the years 2012 and 2015 (included)

### 3. Show the year in which the number of movies released is highest. One highest year is enough

### 4. Show the list of movies such that for each film, the number of actors/actresses is at least five, and the number of genres it belongs to is at most two genres.

### 5. Show the **movies** whose names are longest

### 6. Show the movies whose name contains the word “fighting” (case-insensitive).

### 7. Show the list of distinct genres appearing in the dataset

### 8. List all movies in which the actor Harrison Ford has participated.

### 9. List all movies in which the actors/actresses whose names include the word “Lewis“ (case-insensitive) have participated.

### 10. Show top five actors/actresses that have participated in most movies.

#Exercise 2: RDD-based mainpulation


*   The data is already in one ore more RDDs.
*   You must not convert RDD to DF or use pure Python code.


### 1. Consider a string s that includes only alphabetical letters and spaces. Check whether s is a palindrome (case-insensitive).

### 2. Consider a string s that includes only alphabetical letters and spaces. Check whether s is a pangram (case-insensitive).

#Exercise 3: Frequent patterns and association rules mining

### 0. Load the data file: foodmart.csv


*  A record is a tuple of binary values {0, 1}, each of which denotes the presence of an item (1: bought, 0: not bought).



In [ ]:
!git clone https://github.com/nnthaofit/CSC14118.git
df = spark.read.csv("CSC14118/foodmart.csv", header=True, inferSchema = True)
df.show()

### 1. Convert the given data to the format required by Spark MLlib FPGrowth.

###2.	Apply Spark MLlib FPGrowth to the formatted data. Mine the set of frequent patterns with the minimum support of 0.1. Mine the set of association rules with the minimum confidence of 0.9.

#Exercise 4: Classification

###0. Load the data file: mushroom.csv
*   The data represents a collection of mushroom species.
*   There are 8124 examples, each of which has 22 attributes and it is categorized into either “edible” (e) or “poisonous” (p)


### 1.	Prepare the train and test sets following the ratio 8:2

### 2. Fit a decision tree model on the training set, using Spark MLlib DecisionTreeClassifier with default parameters

### 3. Fit a random forest model on the training set, using Spark MLlib RandomForestClassification with default parameters

### 4. Evaluate the two models on the same test set using the following metrics: areaUnderROC and areaUnderPR

###5. Chain the above steps into a single pipeline

# Exercise 5: Clustering

### 1.	Cluster the data by using Spark MLlib KMeans with k = 2, 3, and 5, using Euclidean distance and cosine distance

### 2. Evaluate each of the above clustering results using silhoutte score. Which configuration yeilds the best clustering?

###3. Chain the above steps into a single pipeline

###4. For each clustering result obtained above, count the number of examples that belong to each of the three species.

## Exercise 6: Network manipulation with Spark GraphFrames

###0. Load the data files: users.txt and followers.txt

###1.	Construct a graph from the given data to demonstrate a tiny social network


###2.	Apply Graphs graphPageRank to the network to obtain a ranking list of users in terms of followers

###3. Find connected components on the graph, using Graphs connectedComponents or stronglyConnectedComponents